<h1 style="font-size: 36px; color: #FFD700">7 - OPTIMIZING-MODEL-PARAMETERS</h1>

Mục tiêu: Tối ưu tham số của models trên dữ liệu của ta. 

Đào tạo mô hình là một quá trình lặp đi lặp lại; trong mỗi lần lặp lại, mô hình đưa ra dự đoán về đầu ra, tính toán lỗi trong dự đoán của nó ( loss ), thu thập các đạo hàm của lỗi liên quan đến các tham số của nó và tối ưu hóa các tham số này bằng cách sử dụng gradient descent. 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Lấy dữ liệu có sẵn từ pytorch
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
# tải dữ liệu vào data loader theo batch
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
# Tạo một mạng 3 lớp
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork()

HYPERPARAMETER

Siêu tham số là các tham số có thể điều chỉnh cho phép kiểm soát quá trình tối ưu hóa mô hình. Các giá trị siêu tham số khác nhau có thể tác động đến tốc độ đào tạo và hội tụ của mô hình

Xác định các hyperparameter sau để đào tạo:
- epochs: số lần lặp lại trên tập dữ liệu
- batch_size: số lượng mẫu dữ liệu được truyền qua mạng trước khi các tham số được cập nhật
- learning_rate: mức độ cập nhật các tham số mô hình ở mỗi batch/epochs. Các giá trị nhỏ hơn tạo ra tốc độ học chậm, trong khi các giá trị lớn có thể dẫn đến hành vi không thể đoán trước trong quá trình đào tạo.

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

OPTIMIZATION LOOP

Sau khi ta thiết lập hyperparameters, ta có thể đào tạo và tối ưu hóa mô hình của mình bằng optimization loop. Mỗi lần lặp lại của optimization loop được gọi là một epoch.

Mỗi epoch bao gồm 2 phần chính:
- Train loop: lặp lại tập dữ liệu đào tạo và cố gắng hội tụ đến các tham số tối ưu.
- Validation/test loop: lặp lại tập dữ liệu kiểm tra để kiểm tra xem hiệu suất mô hình có được cải thiện hay không.

LOSS FUNCTION

Khi được cung cấp một số dữ liệu đào tạo, mạng chưa được đào tạo có khả năng không đưa ra câu trả lời đúng.

=> Hàm mất mát đo lường mức độ không giống nhau của kết quả thu được so với giá trị mục tiêu và đó là hàm mất mát mà ta muốn giảm thiểu trong quá trình đào tạo. Để tính toán mất mát, ta đưa ra dự đoán bằng cách sử dụng các đầu vào của mẫu dữ liệu đã cho và so sánh với giá trị nhãn dữ liệu thực

Các hàm mất mát phổ biến bao gồm nn.MSELoss (Lỗi bình phương trung bình) cho các tác vụ hồi quy và nn.NLLLoss (Khả năng logarit âm) cho phân loại. nn.CrossEntropyLoss kết hợp nn.LogSoftmaxvà nn.NLLLoss

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss() # ta truyền logits đầu ra của models tới nn.CrossEntropyLoss 
# => chuẩn hóa logits và tính toán lỗi dữ đoán

OPTIMIZER

Tối ưu hóa là quá trình điều chỉnh các tham số mô hình để giảm lỗi mô hình trong mỗi bước đào tạo. Tất cả logic tối ưu hóa được đóng gói trong đối tượng optimizer.

In [8]:
# Khởi tạo bằng cách truyền vào các tham số mô hình cần đc đào tạo và siêu tham số learning_rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Trong training loop, quá trình tối ưu hóa diễn ra theo ba bước:
- Gọi optimizer.zero_grad() để thiết lập lại gradient của các tham số mô hình. Theo mặc định, gradient sẽ cộng dồn; để tránh đếm trùng, ta sẽ đặt chúng về 0 một cách rõ ràng ở mỗi lần lặp lại.
- Truyền ngược lại tổn thất dự đoán bằng lệnh gọi đến loss.backward(). PyTorch gửi các gradient của tổn thất liên quan đến từng tham số.
- Khi đã có gradient, chúng ta gọi optimizer.step()để điều chỉnh các tham số theo gradient được thu thập trong quá trình truyền ngược.

Triển khai đầy đủ

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Tổng số mẫu trong tập huấn luyện

    model.train()  # Đặt model về chế độ huấn luyện

    for batch, (X, y) in enumerate(dataloader):
        # Dự đoán và tính loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Lan truyền ngược và cập nhật trọng số
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # In loss mỗi 100 batch
        if batch % 100 == 0:
            loss_value = loss.item()
            current = batch * batch_size + len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    # Đặt mô hình ở chế độ đánh giá (evaluation mode)
    # Giúp tắt dropout, batchnorm hoạt động ổn định hơn
    model.eval()

    size = len(dataloader.dataset)      # Tổng số mẫu test
    num_batches = len(dataloader)       # Tổng số batch
    test_loss, correct = 0, 0           # Biến lưu tổng loss và số mẫu đúng

    # Tắt tính gradient để tiết kiệm bộ nhớ và tăng tốc độ
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)                                 # Dự đoán
            test_loss += loss_fn(pred, y).item()            # Cộng dồn loss
            correct += (pred.argmax(1) == y).float().sum().item()  # Đếm số dự đoán đúng

    # Tính loss trung bình và accuracy toàn tập
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Ta đã tạo Loss_function và optimizer, sau đó truyền nó cho train_loop và test_loop. Ta có thể tăng epoch và batch_size để theo dõi hiệu suất mô hình

In [11]:
loss_fn = nn.CrossEntropyLoss()  # Hàm mất mát dùng cho phân loại nhiều lớp
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # Tối ưu SGD

epochs = 10  # Số lần huấn luyện toàn bộ dữ liệu (epoch)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    # Huấn luyện model với dữ liệu huấn luyện
    train_loop(train_dataloader, model, loss_fn, optimizer)
    
    # Đánh giá model với dữ liệu kiểm tra
    test_loop(test_dataloader, model, loss_fn)

print("Done!")  # In ra khi huấn luyện xong

Epoch 1
-------------------------------
loss: 2.308785  [   64/60000]
loss: 2.291500  [ 6464/60000]
loss: 2.278466  [12864/60000]
loss: 2.272098  [19264/60000]
loss: 2.257041  [25664/60000]
loss: 2.235273  [32064/60000]
loss: 2.240129  [38464/60000]
loss: 2.207500  [44864/60000]
loss: 2.203838  [51264/60000]
loss: 2.176012  [57664/60000]
Test Error: 
 Accuracy: 48.0%, Avg loss: 2.167694 

Epoch 2
-------------------------------
loss: 2.181559  [   64/60000]
loss: 2.170525  [ 6464/60000]
loss: 2.117822  [12864/60000]
loss: 2.126220  [19264/60000]
loss: 2.085720  [25664/60000]
loss: 2.029249  [32064/60000]
loss: 2.058931  [38464/60000]
loss: 1.978039  [44864/60000]
loss: 1.989515  [51264/60000]
loss: 1.918452  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 1.912824 

Epoch 3
-------------------------------
loss: 1.949598  [   64/60000]
loss: 1.920287  [ 6464/60000]
loss: 1.806952  [12864/60000]
loss: 1.837857  [19264/60000]
loss: 1.739152  [25664/60000]
loss: 1.688069  [32064/600